In [12]:
import sys
import os
current_dir = os.getcwd()
parent_parent_dir = os.path.abspath(os.path.join(current_dir, '../..'))

sys.path.append(parent_parent_dir)


In [11]:
from ba_code_project.importable_functions.optics_functions import *
from ba_code_project.import_scripts.get_x_y_tuples import *
from ba_code_project.importable_functions.render_vehicle_tracks import *
import pandas as pd
from sklearn.cluster import OPTICS
from tslearn.metrics import dtw_path_from_metric
import numpy as np
import ast

In [3]:
df_k729_2022_cuid = pd.read_csv('../datasets/k729_2022_cuid_lat_lon.csv')
df_k729_2022_cuid_grouped = pd.read_csv('../datasets/k729_2022_cuid_grouped_lat_lon.csv')
df_k729_2022_cuid_grouped['x'] = df_k729_2022_cuid_grouped['x'].apply(lambda x: ast.literal_eval(x))
df_k729_2022_cuid_grouped['y'] = df_k729_2022_cuid_grouped['y'].apply(lambda y: ast.literal_eval(y))
df_k729_2022_cuid_grouped['vx'] = df_k729_2022_cuid_grouped['vx'].apply(lambda vx: ast.literal_eval(vx))
df_k729_2022_cuid_grouped['vy'] = df_k729_2022_cuid_grouped['vy'].apply(lambda vy: ast.literal_eval(vy))
df_k729_2022_cuid_grouped['lat'] = df_k729_2022_cuid_grouped['lat'].apply(lambda lat: ast.literal_eval(lat))
df_k729_2022_cuid_grouped['lon'] = df_k729_2022_cuid_grouped['lon'].apply(lambda lon: ast.literal_eval(lon))
list_k729_2022_x_y_tuples = get_x_y_tuple_list(df_k729_2022_cuid_grouped, ['x','y'])
k729_2022_vx_vy_tuples = get_x_y_tuple_list(df_k729_2022_cuid_grouped, ['vx','vy'])
k729_2022_lat_lon_tuples = get_x_y_tuple_list(df_k729_2022_cuid_grouped,['lat','lon'])

In [4]:
def calculate_dtw_distance_matrix(data:list[list], **dtw_metrics):
    '''

    :param data: List of lists, where each sublist contains list-entries of x-y tuples, e.g. [sublist_1,...,sublist_n] = [[[x_11,y_11],...,[x_1m,y_1m]],...,[[x_n1,y_n1],...,[x_nk,y_nk]]]
    :param **dtw_metrics: One or multiple of the following: `sakoe_chiba_radius`, `metric`, `itakura_max_slope`, `global_constraint`
    '''
    n = len(data)
    dtw_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1,n):
            path, distance  = dtw_path_from_metric(data[i], data[j], **dtw_metrics)
            dtw_matrix[i][j] = distance
            dtw_matrix[j][i] = distance

    return dtw_matrix

In [5]:
dtw_distance_matrix = calculate_dtw_distance_matrix(k729_2022_lat_lon_tuples)

In [8]:
%%capture
run_no = 6
min_samples = 6
max_eps = 0.0005
metric='precomputed'
cluster_method = 'dbscan'
xi = 0.05
if cluster_method != 'dbscan':
    metrics_description = f'with min samples: {min_samples}\nmax_eps: {max_eps}\nmetric: DTW\nclustermethod: {cluster_method}\nxi: {xi}'
else:
    metrics_description = f'with min samples: {min_samples}\nmax_eps: {max_eps}\nmetric: DTW\nclustermethod: {cluster_method}'


optics_clustering = OPTICS(min_samples=min_samples,
                           max_eps=max_eps,
                           cluster_method=cluster_method,
                           metric=metric,
                           ).fit(dtw_distance_matrix)

clusters = get_clusters_from_optics(optics_clustering.labels_)


render_vehicle_tracks_clusters_lat_lon(df_k729_2022_cuid,
                                       df_k729_2022_cuid_grouped,
                                       clusters,
                                       'Clusters w/ OPTICS', 
                                       metrics_description,
                                       f'optics_run_{run_no}_{cluster_method}',
                                       'k729_2022_lat_lon',
                                       f'clustering/optics/')